In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

# 各種バリアントの可視化
gbk.transcript_variants()

In [ ]:
# バリアントのリストを作成 
var = gbk.get_mrna_ids()

# 特殊なバリアントを除外する
var.remove("XM_024453100.2")
var

In [ ]:
# すべてのエクソンの配列をvalueとする辞書を作成
# エクソン番号は、バリアントに関わらず統一的なナンバリングにする

all_exon_list = []
for i in var:
    gbk.set_mrna_id(i)
    for j in gbk.exon_list():
        if j not in all_exon_list:
            all_exon_list.append(j)

all_exon_list.sort(key=lambda x: x[0])

exon_dic = {}
for i in range(len(all_exon_list)):
    exon_dic[i+1] = all_exon_list[i]

In [ ]:
# 各バリンアントのおける統一的エクソンナンバーの組み合わせを作成
def get_key(value):
    for i in exon_dic.keys():
        if exon_dic[i] == value:
            return i

variant_dic = {}
for v in var:
    gbk.set_mrna_id(v)
    tmp_list = []
    for i in gbk.exon_list():
            tmp_list.append(get_key(i))
    variant_dic[v] = tmp_list


In [ ]:
# 存在しうるエクソンの組み合わせを作成
exon_comb = []
for i in var:
    ls = variant_dic[i]
    for i in range(len(ls)-1):
        exon_comb.append((ls[i], ls[i+1]))
exon_comb = set(exon_comb)

In [ ]:
# exon40からexon230までの範囲に解析を限定する
exon_comb_ltd = []
for i,j in exon_comb:
    if i >= 40 and j <= 230:
        exon_comb_ltd.append((i,j))

exisistent_comb = sorted(exon_comb_ltd)
print("exon_comb_ltdの要素数:", len(exisistent_comb))
# exon_comb_ltd

In [ ]:
#  特殊なスプライスを受けるエクソン
for i in exon_dic.keys():
    if i < len(exon_dic.keys())-1:
        if exon_dic[i][1] > exon_dic[i+1][0]:
            print(i)

In [ ]:
# エクソンを一つ以上飛ばす組み合わせを作成
exon_distant_comb = []
for i,j in exisistent_comb:
    if j - i > 1:
        exon_distant_comb.append((i,j))
# exon_distant_comb

In [ ]:
space_filling_comb = []
for i,j in exon_distant_comb:
    for k in range(j-i):
        space_filling_comb.append((i,i+k+1))

space_filling_comb = set(space_filling_comb)
# space_filling_comb

In [ ]:
non_exisistent_comb = []
for i in space_filling_comb:
    if i not in exisistent_comb:
        non_exisistent_comb.append(i)
non_exisistent_comb = sorted(non_exisistent_comb)

In [ ]:
# エクソンの組み合わせがあるイントロンの始点と終点のリスト
exisistent_comb_region = []
for i,j in exisistent_comb:
    exisistent_comb_region.append((exon_dic[i][1], exon_dic[j][0]))
# exisistent_comb_region

# エクソンの組み合わせが無いイントロンの始点と終点のリスト
non_exisistent_comb_region = []
for i,j in non_exisistent_comb:
    non_exisistent_comb_region.append((exon_dic[i][1], exon_dic[j][0]))
# non_exisistent_comb_region

In [ ]:
# TTN遺伝子全長配列
seq = str(gbk.gDNA_seq())

In [ ]:
# exisistent_comb_regionのイントロン両端のn塩基の配列を取得
n = 15

exisistent_comb_region_edges = []
for i,j in exisistent_comb_region:
    left = seq[i:i+n]
    right = seq[j-n:j]
    exisistent_comb_region_edges.append((left+right))
exisistent_comb_region_edges

# non_exisistent_comb_regionのイントロン両端のn塩基の配列を取得
non_exisistent_comb_region_edges = []
for i,j in non_exisistent_comb_region:
    left = seq[i:i+n]
    right = seq[j-n:j]
    non_exisistent_comb_region_edges.append((left+right))
non_exisistent_comb_region_edges